In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing libaries: 
#libaries for ploting data 
import seaborn as sns
import matplotlib.pyplot as plt
#regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor


#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression

In [ ]:
# Reading the train and test data as a dataframe.
train_df = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv",parse_dates = ["datetime"])
test_df = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv",parse_dates = ["datetime"])

In [ ]:
# Displaying the first 5 rows of the train and test data using the head method.
display(train_df.head())
display(test_df.head())

In [ ]:
display(train_df.info())

In [ ]:
display(test_df.info())

In [ ]:
# checking if there is any missing values in the trian data 
display(train_df.isnull().any())  
### YaaaYY NO missing values :D ### 

In [ ]:
#checking if there is a missing values in the test data
display(test_df.isnull().any()) 
### YaaaYY NO missing values :D ### 

In [ ]:
display(train_df.describe().T)
display(test_df.describe().T)

In [ ]:
#checking id there is any duplicated row is my train and test data.
display(train_df.duplicated().sum()) 
display(train_df.duplicated().sum()) 
### YaaaYY .. No Duplicates ;) ###

In [ ]:
train_df["year"] = train_df["datetime"].dt.year
train_df["hour"] = train_df["datetime"].dt.hour
train_df["dayofweek"] = train_df["datetime"].dt.dayofweek

test_df["year"] = test_df["datetime"].dt.year
test_df["hour"] = test_df["datetime"].dt.hour
test_df["dayofweek"] = test_df["datetime"].dt.dayofweek

In [ ]:
display(train_df.head())
display(test_df.head())

In [ ]:
sns.histplot(data=train_df, x='count', hue='season', kde=True)

In [ ]:
sns.boxplot(x='season', y='count', data=train_df)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
corr_matrix = train_df.corr().abs()
sns.heatmap(corr_matrix, annot=True, linewidths=.5)

In [ ]:
# feature that has high correlation with the "count" column are :
# temp = 0.39 , atemp=0.39 , humidity = 0.32 , season = 0.16 weather = 0.13 

In [ ]:


# train_df.drop(["datetime", "windspeed", "casual", "registered"], 1, inplace=True)
# test_df.drop(["datetime", "windspeed", ], 1, inplace=True
test_date = test_df['datetime']
train_df.drop(["datetime", "casual", "registered"], 1, inplace=True)
test_df.drop(["datetime"], 1, inplace=True)

In [ ]:
train_df.head()

In [ ]:
Y = train_df['count']
x = train_df.drop(columns=['count'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_val, y_train, y_val = train_test_split(x,Y, test_size=0.2, random_state=42)

In [ ]:
models=[RandomForestRegressor(),AdaBoostRegressor(),BaggingRegressor(),SVR(),KNeighborsRegressor()]
model_names=['RandomForestRegressor','AdaBoostRegressor','BaggingRegressor','SVR','KNeighborsRegressor']
rmsle=[]
d={}
for model in range (len(models)):
    clf=models[model]
    clf.fit(X_train,y_train)
    test_pred=clf.predict(X_val)
    rmsle.append(mean_squared_log_error(test_pred,y_val))
d={'Modelling Algo':model_names,'RMSLE':rmsle}   
d

rmsle_frame=pd.DataFrame(d)
rmsle_frame

In [ ]:
# model=BaggingRegressor(n_estimators= 7, bootstrap = False, max_samples= 0.75,random_state=43)
# model=BaggingRegressor(max_samples=0.3)
# model=BaggingRegressor(base_estimator=XGBRegressor(),n_estimators= 7)
# model=RandomForestRegressor()
model=BaggingRegressor()
model.fit(X_train,y_train)

In [ ]:
test_pred = model.predict(X_train)
rmsle = mean_squared_log_error(test_pred,y_train)
print("The Error for the train data is {}".format(rmsle))

In [ ]:
test_pred = model.predict(X_val)
rmsle = mean_squared_log_error(test_pred,y_val)
print("The Error for the validation data is {}".format(rmsle))

In [ ]:
y_pred = model.predict(test_df)

In [ ]:
output = pd.DataFrame({'datetime': test_date,
                       'count': y_pred})
output.to_csv('submission.csv', index=False)